In [2]:
# will create NN to get weight we can then copy other to the SNN small demo

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

# --- Load and prepare data ---
iris = load_iris()
X = iris['data']
y = iris['target']

# Normalize input to [0, 1]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# One-hot encode output
y_cat = to_categorical(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# --- Custom activation function ---
def directional_push_activation(w=2.0):
    def activation(x):
        # Clip to avoid divide-by-zero
        x = tf.clip_by_value(x, 1e-6, 1.0 - 1e-6)
        return 1 / (1 + tf.pow((1 - x) / x, tf.exp(w / 3)))
    return activation

# Choose your activation weight
activation_weight = 2.0

# --- Build model ---
inputs = Input(shape=(4,))
x = Dense(10, activation=directional_push_activation(activation_weight))(inputs)
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# --- Train model ---
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

# --- Evaluate ---
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

# --- Copy weights ---
weights = model.get_weights()

# If you want to use them elsewhere:
# model.set_weights(weights_copy)

# Optional: print layer weights
for i, w in enumerate(weights):
    print(f"Layer {i} weights shape: {w.shape}")


Test Accuracy: 0.9000
Layer 0 weights shape: (4, 10)
Layer 1 weights shape: (10,)
Layer 2 weights shape: (10, 3)
Layer 3 weights shape: (3,)


In [4]:

my_input = np.array([[5.1, 3.5, 1.4, 0.2]])  # sepal length, sepal width, petal length, petal width

# need to standardize the inputs to between 0 and 1.0 ^ 


# Scale it using the same scaler
my_input_scaled = scaler.transform(my_input)

# Predict
prediction = model.predict(my_input_scaled)
predicted_class = np.argmax(prediction)

print(f"Predicted probabilities: {prediction}")
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted probabilities: [[0.94778425 0.03397965 0.01823603]]
Predicted class: 0


In [5]:
print(weights)

print(X[1])

[array([[ 0.1135968 ,  0.18025309,  0.10094303, -0.5609372 , -0.19758941,
        -0.20204231, -0.3657724 ,  0.02413964, -0.6838493 ,  0.13974035],
       [-0.4299119 ,  0.6738531 ,  0.9506301 , -0.18848935,  0.14345719,
        -0.48180234,  0.00865681,  0.01765617,  0.17112266, -0.08891857],
       [-0.09449631, -0.8266945 , -1.235361  ,  0.14157972,  0.51368946,
         0.321979  , -0.1708726 ,  0.07350195, -0.24007349, -0.5640322 ],
       [-0.08869338, -0.745832  , -0.36420307,  0.36688203,  0.85630447,
        -0.5828457 , -0.5360476 ,  0.82350487, -0.12507023, -0.4661583 ]],
      dtype=float32), array([ 0.        ,  0.5621115 ,  0.49492553, -0.12487811, -0.01794191,
        0.        , -0.00530157, -0.11453775, -0.06169549,  0.        ],
      dtype=float32), array([[ 0.47116745, -0.64527065, -0.49626732],
       [ 0.7116071 , -1.1385437 , -1.088651  ],
       [ 0.43341887, -1.4892002 , -1.3539809 ],
       [ 0.28178445,  0.152175  , -0.54456943],
       [-1.5054257 ,  0.12284